In [ ]:
import glob
import os
import unicodedata
import string
import time
import math
import random
import torch
import torch.nn as nn


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [ ]:


def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# Load data: each .txt file is a category
category_lines = {}
all_categories = []

In [7]:

for filename in glob.glob("./data/names/*.txt"):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines



In [9]:

n_categories = len(all_categories)
if n_categories == 0:
    raise RuntimeError("No data found! Download the names dataset from PyTorch tutorial.")

print("Categories:", all_categories)
print("Total categories:", n_categories)

Categories: ['french', 'german', 'next_word_predictor']
Total categories: 3


In [10]:
def letterToIndex(letter):
    return all_letters.find(letter)

def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

In [11]:


def categoryToTensor(category):
    idx = all_categories.index(category)
    return torch.tensor([idx], dtype=torch.long)


In [12]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        hidden = torch.zeros(1, 1, self.hidden_size)
        for i in range(input.size(0)):
            out, hidden = self.rnn(input[i].unsqueeze(0), hidden)
        output = self.h2o(hidden.squeeze(0))
        return self.softmax(output)

In [13]:
def randomTrainingExample():
    category = random.choice(all_categories)
    line = random.choice(category_lines[category])
    category_tensor = categoryToTensor(category)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor


In [14]:

def train(model, criterion, optimizer, n_iters=10000, print_every=500):
    current_loss = 0
    all_losses = []

    for iter in range(1, n_iters + 1):
        category, line, category_tensor, line_tensor = randomTrainingExample()
        optimizer.zero_grad()
        output = model(line_tensor)
        loss = criterion(output, category_tensor)
        loss.backward()
        optimizer.step()

        current_loss += loss.item()

        if iter % print_every == 0:
            guess = all_categories[torch.argmax(output).item()]
            correct = '✓' if guess == category else f'✗ ({category})'
            print(f'[{iter}] Loss: {loss.item():.4f}  {line} → {guess} {correct}')
            all_losses.append(current_loss / print_every)
            current_loss = 0

    return all_losses


In [15]:
n_hidden = 128
rnn = CharRNN(n_letters, n_hidden, n_categories)

criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.005)

print("Training started...")
losses = train(rnn, criterion, optimizer, n_iters=5000, print_every=500)

print("\nTraining complete.")

Training started...
[500] Loss: 1.2557  Paul Two years. → french ✗ (next_word_predictor)
[1000] Loss: 0.8726        en science des materiaux et en biologie. Aujourd'hui, l'exploration spatiale  → german ✗ (french)
[1500] Loss: 0.5278   analytische Philosophie, die sich auf Logik und Sprache konzentrierte. Auch heute noch  → german ✓
[2000] Loss: 0.7848   Hegels, der Materialismus von Marx, der Existentialismus von Kierkegaard, Nietzsche, Sartre  → french ✗ (german)
[2500] Loss: 0.9026        Internationale ISS, a permis aux humains de vivre et de travailler dans  → german ✗ (french)
[3000] Loss: 0.0644   globalisierten Welt zu suchen. Sie ist kein abgeschlossenes System von Antworten, sondern  → german ✓
[3500] Loss: 0.3365   → next_word_predictor ✓
[4000] Loss: 0.3908  Chandler Well, what What What is it That she left you That she likes women That she left you for another woman that likes women → next_word_predictor ✓
[4500] Loss: 1.5253   fur kritisches Denken, wahrend Platons Ideenl

In [17]:
def predict(input_line):
    with torch.no_grad():
        output = rnn(lineToTensor(input_line))
        category_idx = torch.argmax(output).item()
        return all_categories[category_idx]

test_names = [ "Pierre", "André", "François", "Laurent","Claude","Michel","René"]
for name in test_names:
    print(f'{name:15} → {predict(name)}')


Pierre          → french
André           → next_word_predictor
François        → next_word_predictor
Laurent         → french
Claude          → french
Michel          → french
René            → german


In [18]:
import torch

def evaluate_model(model, data, n_samples=500):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for _ in range(n_samples):
            category, line, category_tensor, line_tensor = randomTrainingExample()
            output = model(line_tensor)
            pred = torch.argmax(output).item()
            if pred == category_tensor.item():
                correct += 1
            total += 1
    
    acc = 100 * correct / total
    print(f"Accuracy on random samples: {acc:.2f}%")
    return acc

evaluate_model(rnn, category_lines)


Accuracy on random samples: 64.20%


64.2